# W209 Final Project

## Data Processing

We will be using two sets of data from a current Kaggle competition, “Two Sigma: Using News to Predict Stock Movements.” The first is stock market data provided by Intrinion (from 2007 to present), which contains financial market information such as opening & closing prices, trade volumes, calculated returns, etc. The second data set is news data provided by Thomson Reuters, for the same time period. This data set has information regarding news articles and alerts (article headlines, sentiments, etc.) that have been posted about the companies included in the market data set. The two data sets can be joined using the unique assetCode or assetName (i.e. company name) identifiers as well as the time variable. It is important to note that one company can have more than one assets, which will be taken into account in our visualization. 

Link to data souce: https://www.kaggle.com/c/two-sigma-financial-news/data    

Note: In order to download the full dataset from kaggle we had to write a kernel within the kaggle competition and export the data to our local machines. 


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import time

In [44]:
# reading in csv without index column 0

start = time.time()

market_df = pd.read_csv('data/market_data.csv', index_col=0)
news_df = pd.read_csv('data/news_data.csv', index_col=0)

end = time.time() - start

print("Data import completed in {}s".format(end))

C:\Users\kw.UNLOVEDPC\Anaconda3\envs\py3-Env\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Data import completed in 82.29488706588745s


In [37]:
# checking

#market_df.head()
#news_df.head()

In [45]:
# parsing the time columns in both df
start = time.time()

market_df['time'] = pd.to_datetime(market_df['time'])
news_df['time'] = pd.to_datetime(news_df['time'])

end = time.time() - start

print("Date parse completed in {}s".format(end))

Date parse completed in 15.388837814331055s


In [46]:
# adding a date column at index 0

market_df.insert(0, 'date', market_df['time'].apply(lambda x: x.date()))
news_df.insert(0, 'date', news_df['time'].apply(lambda x: x.date()))

In [40]:
market_df.columns.values

array(['date', 'time', 'assetCode', 'assetName', 'volume', 'close',
       'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'universe'], dtype=object)

In [41]:
news_df.columns.values

array(['date', 'time', 'sourceTimestamp', 'firstCreated', 'sourceId',
       'headline', 'urgency', 'takeSequence', 'provider', 'subjects',
       'audiences', 'bodySize', 'companyCount', 'headlineTag',
       'marketCommentary', 'sentenceCount', 'wordCount', 'assetCodes',
       'assetName', 'firstMentionSentence', 'relevance', 'sentimentClass',
       'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
       'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
       'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D',
       'volumeCounts12H', 'volumeCounts24H', 'volumeCounts3D',
       'volumeCounts5D', 'volumeCounts7D'], dtype=object)

In [48]:
# dropping unused columns

news_drop_cols =['sourceTimestamp', 'firstCreated', 'sourceId', 'headline', 
                 'urgency', 'takeSequence', 'provider', 'audiences', 'bodySize', 
                 'companyCount', 'headlineTag', 'marketCommentary', 'sentenceCount', 
                 'firstMentionSentence', 'noveltyCount12H', 'noveltyCount24H', 
                 'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
                 'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D']
news_df.drop(news_drop_cols, axis=1, inplace=True)


In [ ]:
assetCode_list = market2016_df['assetCode'].unique()

In [43]:
# cleaning up assetCode in news_df

start = time.time()

# for each unique assetCode in the market dataframe, find and replace in the news dataframe
# keeping the parenthesis '' in the search parameter

for c in assetCode_list:
    mask = news_df['assetCodes'].str.contains('\''+c+'\'', regex=False)
    news_df.loc[mask, 'assetCodes'] = c

# for those lines in news which do not have a match to a unique assetCode in market dataframe
# they will still be in the format {'600115.SS', '0670.HK', 'CEA.N'}
# hence match '{' and remove those lines from the news dataframe

mask_rm = news_df['assetCodes'].str.contains('{', regex=False)
news_df.drop(news2016_df.loc[mask_rm, 'assetCodes'].index, axis=0, inplace=True)

# reset the index for news dataframe
news_df.reset_index(drop=True, inplace=True)

end = time.time() - start

print("Asset Code cleanup completed in {}s".format(end))

Asset Code cleanup completed in 4160.873142719269s


In [44]:
# renaming the news dataframe column from 'assetCodes' to 'assetCode'
news_df.rename(columns = {'assetCodes':'assetCode'}, inplace = True)

# merging the two dataframes on data and assetCode
combined_df = market_df.merge(news_df, how='left', on=['date','assetCode'])

In [45]:
# saving to csv

news_df.to_csv('data/news_data_cleaned.csv')
combined_df.to_csv('data/combined_data.csv')

In [48]:
# slicing out only 2016 data for exploration

combined_df['date'] = pd.to_datetime(combined_df['date'])

combined2016_df = combined_df[(combined_df['date'] >= '2016-01-01') & (combined_df['date'] < '2017-01-01')]

In [50]:
# checking

#combined2016_df.head()
#combined2016_df.tail()

In [52]:
combined2016_df.to_csv('data/combined2016_data.csv')

In [ ]:
# switching over to Tableau...

In [2]:
# reading in csv without index column 0

start = time.time()

combined_df = pd.read_csv('data/combined_data.csv', index_col=0)
end = time.time() - start

print("Data import completed in {}s".format(end))

C:\Users\kw.UNLOVEDPC\Anaconda3\envs\py3-Env\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Data import completed in 55.83366274833679s


In [11]:
list(combined_df)

['date',
 'time_x',
 'assetCode',
 'assetName_x',
 'volume',
 'close',
 'open',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevMktres1',
 'returnsOpenPrevMktres1',
 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'returnsOpenNextMktres10',
 'universe',
 'time_y',
 'subjects',
 'wordCount',
 'assetName_y',
 'relevance',
 'sentimentClass',
 'sentimentNegative',
 'sentimentNeutral',
 'sentimentPositive',
 'sentimentWordCount']

In [25]:
cd2 = combined_df[combined_df.assetName_x != combined_df.assetName_y]

In [53]:
cd2.groupby(['assetName_y', 'assetName_x', 'assetCode']).count().head(100)

,,,date,time_x,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,...,universe,time_y,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount
assetName_y,assetName_x,assetCode,,,,,,,,,,,,,,,,,,,,,
ADESA Inc,KAR Auction Services Inc,KAR.N,39,39,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
ADT Corp,ADT Inc,ADT.N,721,721,721,721,721,721,721,714,714,721,...,721,721,721,721,721,721,721,721,721,721
AGCO Corp,First Majestic Silver Corp,AG.N,326,326,326,326,326,326,326,324,324,326,...,326,326,326,326,326,326,326,326,326,326
Advanced Semiconductor Engineering Inc,ASE Technology Holding Co Ltd,ASX.N,818,818,818,818,818,818,818,817,817,818,...,818,818,818,818,818,818,818,818,818,818
Allergan Inc,Allergan plc,AGN.N,3721,3721,3721,3721,3721,3721,3721,3700,3700,3721,...,3721,3721,3721,3721,3721,3721,3721,3721,3721,3721
Ameron International Corp,AMN Healthcare Services Inc,AMN.N,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
Anheuser Busch Inbev SA,Anheuser Busch Inbev NV,BUD.N,13755,13755,13755,13755,13755,13755,13755,13734,13734,13755,...,13755,13755,13755,13755,13755,13755,13755,13755,13755,13755
Anheuser-Busch Companies LLC,Anheuser Busch Inbev NV,BUD.N,1426,1426,1426,1426,1426,1426,1426,1408,1408,1426,...,1426,1426,1426,1426,1426,1426,1426,1426,1426,1426
Ashland LLC,Ashland Global Holdings Inc,ASH.N,2042,2042,2042,2042,2042,2042,2042,2016,2016,2042,...,2042,2042,2042,2042,2042,2042,2042,2042,2042,2042


In [111]:
# Cleaning data, correcting assetNames

combined_df.assetName_y.replace(to_replace='ADESA Inc', value='KAR Auction Services Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='ADT Corp', value='ADT Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Advanced Semiconductor Engineering Inc', value='ASE Technology Holding Co Ltd', inplace=True)
combined_df.assetName_y.replace(to_replace='Allergan Inc', value='Allergan plc', inplace=True)
combined_df.assetName_y.replace(to_replace='Ameron International Corp', value='AMN Healthcare Services Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Anheuser Busch Inbev SA', value='Anheuser Busch Inbev NV', inplace=True)
combined_df.assetName_y.replace(to_replace='Anheuser-Busch Companies LLC', value='Anheuser Busch Inbev NV', inplace=True)
combined_df.assetName_y.replace(to_replace='Ashland LLC', value='Ashland Global Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Atlassian Corporation PLC', value='Atlassian Corp PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='BancorpSouth Inc', value='BancorpSouth Bank', inplace=True)
combined_df.assetName_y.replace(to_replace='CBOE Holdings Inc', value='Cboe Global Markets Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='CenterState Banks Inc', value='CenterState Bank Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Chubb Corp', value='Chubb Ltd', inplace=True)
combined_df.assetName_y.replace(to_replace='Coca-Cola Enterprises Inc', value='Coca-Cola European Partners PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Coeur d\'Alene Mines Corp', value='Coeur Mining Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Cogent Communications Group Inc', value='Cogent Communications Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='CommVault Systems Inc', value='Commvault Systems Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Dana Corp', value='Dana Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Delphi Automotive PLC', value='Delphi Technologies PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Dennys Corp', value='Denny\'s Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Discovery Communications Inc', value='Discovery Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Dominion Resources Inc', value='Dominion Energy Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Energy Transfer LP', value='Energy Transfer Partners LP', inplace=True)
combined_df.assetName_y.replace(to_replace='Enterprise GP Holdings LP', value='EP Energy Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Envision Healthcare Holdings Inc', value='Envision Healthcare Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Evercore Partners Inc', value='Evercore Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Expedia Inc', value='Expedia Group Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Express Scripts Inc', value='Express Scripts Holding Co', inplace=True)
combined_df.assetName_y.replace(to_replace='FMC Technologies Inc', value='TechnipFMC PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Genworth Holdings Inc', value='Genworth Financial Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Gerdau Macsteel Inc', value='Quanex Building Products Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='HCA Holdings Inc', value='HCA Healthcare Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Herbalife Ltd', value='Herbalife Nutrition Ltd', inplace=True)
combined_df.assetName_y.replace(to_replace='ITT Corp', value='ITT Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='J2 Global Inc', value='j2 Global Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Jabil Circuit Inc', value='Jabil Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Johnson Controls Inc', value='Johnson Controls International PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Kinder Morgan Kansas Inc', value='Kinder Morgan Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Knight Transportation Inc', value='Knight-Swift Transportation Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='LSI Corp', value='Life Storage Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='MGOC Inc', value='Media General Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Metlife Inc', value='MetLife Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Monro Muffler Brake Inc', value='Monro Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Monster Beverage 1990 Corp', value='Monster Beverage Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Mylan Inc', value='Mylan NV', inplace=True)
combined_df.assetName_y.replace(to_replace='NextEra Energy Inc', value='Nextera Energy Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Nicor Inc', value='Southern Company Gas', inplace=True)
combined_df.assetName_y.replace(to_replace='Open Text Corp Canada', value='Open Text Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='PACCAR Inc', value='Paccar Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Parkway Properties Inc', value='Parkway Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Pentair plc', value='Pentair PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Potlatch Corp', value='PotlatchDeltic Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Priceline Group Inc', value='Booking Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Quintiles Transnational Holdings Inc', value='Quintiles IMS Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Reliance Steel and Aluminum Co', value='Reliance Steel & Aluminum Co', inplace=True)
combined_df.assetName_y.replace(to_replace='Ritchie Bros Auctioneers Inc', value='Ritchie Bros. Auctioneers Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Rowan Companies Inc', value='Rowan Companies PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='SAP AG', value='SAP SE', inplace=True)
combined_df.assetName_y.replace(to_replace='Salesforce.Com Inc', value='Salesforce.com Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Sensata Technologies Holding NV', value='Sensata Technologies Holding PLC', inplace=True)
combined_df.assetName_y.replace(to_replace='Sohu.Com Inc', value='Sohu.com Ltd', inplace=True)
combined_df.assetName_y.replace(to_replace='Southwest Gas Corp', value='Southwest Gas Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Store Capital Corp', value='STORE Capital Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Sun Healthcare Group Inc (Pre-Merger)', value='Sabra Health Care REIT Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Sunoco Inc', value='Sunoco LP', inplace=True)
combined_df.assetName_y.replace(to_replace='Swift Transportation Co', value='Knight-Swift Transportation Holdings Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='TAL International Group Inc', value='TAL Education Group', inplace=True)
combined_df.assetName_y.replace(to_replace='TRI Pointe Homes Inc', value='TRI Pointe Group Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Take Two Interactive Software Inc', value='Take-Two Interactive Software Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Telekomunikasi Indonesia (Persero) Tbk PT', value='Telekomunikasi Indonesia (Persero) Tbk Perusahaan Perseroan PT', inplace=True)
combined_df.assetName_y.replace(to_replace='TiVo Solutions Inc', value='TiVo Corp', inplace=True)
combined_df.assetName_y.replace(to_replace='Transocean Inc', value='Transocean Ltd', inplace=True)
combined_df.assetName_y.replace(to_replace='Trimble Navigation Ltd', value='Trimble Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Ulta Salon Cosmetics and Fragrance Inc', value='Ulta Beauty Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='W. P. Carey Inc', value='WP Carey Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='WMI Holdings Corp', value='Waste Management Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Wal-Mart Stores Inc', value='Walmart Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Walgreen Co', value='Walgreens Boots Alliance Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Waste Connections US Inc', value='Waste Connections Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Western Alliance Bancorporation', value='Western Alliance Bancorp', inplace=True)
combined_df.assetName_y.replace(to_replace='WisdomTree Investments Inc', value='Wisdom Tree Investments Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Yahoo! Inc', value='Altaba Inc', inplace=True)
combined_df.assetName_y.replace(to_replace='Zillow Inc', value='Zillow Group Inc', inplace=True)

#Cleaning more complicated dual tickers / mergers
combined_df.loc[combined_df.assetCode == 'HRI.N', 'assetName_x'] = 'Herc Holdings Inc'
combined_df.loc[combined_df.assetCode == 'HRI.N', 'assetName_y'] = 'Herc Holdings Inc'
combined_df.loc[combined_df.assetCode == 'HTZ.N', 'assetName_x'] = 'Hertz Global Holdings Inc'
combined_df.loc[combined_df.assetCode == 'HTZ.N', 'assetName_y'] = 'Hertz Global Holdings Inc'

combined_df.loc[combined_df.assetCode == 'BSAC.N', 'assetName_x'] = 'Banco Santander-Chile'
combined_df.loc[combined_df.assetCode == 'BSAC.N', 'assetName_y'] = 'Banco Santander-Chile'
combined_df.loc[combined_df.assetCode == 'SAN.N', 'assetName_x'] = 'Banco Santander SA'
combined_df.loc[combined_df.assetCode == 'SAN.N', 'assetName_y'] = 'Banco Santander SA'

combined_df.loc[combined_df.assetCode == 'TLRD.N', 'assetName_x'] = 'Tailored Brands Inc'
combined_df.loc[combined_df.assetCode == 'TLRD.N', 'assetName_y'] = 'Tailored Brands Inc'
combined_df.loc[combined_df.assetCode == 'MW.N', 'assetName_x'] = 'Men\'s Wearhouse Inc'
combined_df.loc[combined_df.assetCode == 'MW.N', 'assetName_y'] = 'Men\'s Wearhouse Inc'

In [113]:
# Remaining problematic data to be deleted:
combined_df[combined_df.assetName_x != combined_df.assetName_y].groupby(['assetName_y', 'assetName_x', 'assetCode']).count().head(100)

,,,date,time_x,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,...,universe,time_y,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount
assetName_y,assetName_x,assetCode,,,,,,,,,,,,,,,,,,,,,
AGCO Corp,First Majestic Silver Corp,AG.N,326,326,326,326,326,326,326,324,324,326,...,326,326,326,326,326,326,326,326,326,326
Circuit City Stores Inc,Chemours Co,CC.N,696,696,696,696,696,696,696,696,696,696,...,696,696,696,696,696,696,696,696,696,696
National Semiconductor Corp,Nationstar Mortgage Holdings Inc,NSM.N,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312,...,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312
Penn Virginia GP Holdings LP,Pretium Resources Inc,PVG.N,92,92,92,92,92,92,92,89,89,92,...,92,92,92,92,92,92,92,92,92,92
Qwest Communications International Inc,Quintiles IMS Holdings Inc,Q.N,1658,1658,1658,1658,1658,1658,1658,1656,1656,1658,...,1658,1658,1658,1658,1658,1658,1658,1658,1658,1658
Rare Hospitality International Inc,Ultragenyx Pharmaceutical Inc,RARE.O,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
Realogy Group LLC,Hyatt Hotels Corp,H.N,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Spectra Energy Corp,Sea Ltd,SE.N,2502,2502,2502,2502,2502,2502,2502,2501,2501,2502,...,2502,2502,2502,2502,2502,2502,2502,2502,2502,2502
United Airlines Inc,Caleres Inc,CAL.N,3286,3286,3286,3286,3286,3286,3286,3275,3275,3286,...,3286,3286,3286,3286,3286,3286,3286,3286,3286,3286


In [136]:
# Deleting the news related columns in these rows

news_cols_del = ['time_y',
                 'subjects',
                 'wordCount',
                 'assetName_y',
                 'relevance',
                 'sentimentClass',
                 'sentimentNegative',
                 'sentimentNeutral',
                 'sentimentPositive',
                 'sentimentWordCount']

combined_df.loc[(combined_df.assetCode == 'AG.N') & (combined_df.assetName_y == 'AGCO Corp'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'CC.N') & (combined_df.assetName_y == 'Circuit City Stores Inc'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'NSM.N') & (combined_df.assetName_y == 'National Semiconductor Corp'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'PVG.N') & (combined_df.assetName_y == 'Penn Virginia GP Holdings LP'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'Q.N') & (combined_df.assetName_y == 'Qwest Communications International Inc'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'RARE.O') & (combined_df.assetName_y == 'Rare Hospitality International Inc'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'H.N') & (combined_df.assetName_y == 'Realogy Group LLC'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'SE.N') & (combined_df.assetName_y == 'Spectra Energy Corp'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'CAL.N') & (combined_df.assetName_y == 'United Airlines Inc'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'WMS.N') & (combined_df.assetName_y == 'WMS Industries Inc'), news_cols_del] = np.NaN
combined_df.loc[(combined_df.assetCode == 'CCU.N') & (combined_df.assetName_y == 'iHeartCommunications Inc'), news_cols_del] = np.NaN


In [137]:
combined_df[combined_df.assetName_x != combined_df.assetName_y].groupby(['assetName_y', 'assetName_x', 'assetCode']).count().head(100)

,,,date,time_x,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,...,universe,time_y,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount
assetName_y,assetName_x,assetCode,,,,,,,,,,,,,,,,,,,,,


In [138]:
# Dropping the assetName_y, renaming assetName_x to assetName

combined_drop_cols = ['returnsClosePrevMktres1','returnsOpenPrevMktres1', 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'returnsOpenNextMktres10','assetName_y']

combined_df.drop(combined_drop_cols, axis=1, inplace=True)
combined_df.rename(columns = {'assetName_x':'assetName'}, inplace = True)
combined_df.rename(columns = {'time_x':'timeMkt'}, inplace = True)
combined_df.rename(columns = {'time_y':'timeNews'}, inplace = True)

In [172]:
# Calculating the new combined sentimentRating column

combined_df.insert(loc=20, column='sentimentRating', value=np.NaN)
combined_df.loc[(combined_df.sentimentClass == -1), 'sentimentRating'] = -1* combined_df[combined_df.sentimentClass == -1].sentimentNegative
combined_df.loc[(combined_df.sentimentClass == 0), 'sentimentRating'] = 0
combined_df.loc[(combined_df.sentimentClass == 1), 'sentimentRating'] = combined_df[combined_df.sentimentClass == 1].sentimentPositive

In [176]:
combined_df.head(30)

,date,timeMkt,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,...,timeNews,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
0,2007-02-01,2007-02-01 22:00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,-0.001860,...,2007-02-01 16:00:04,"{'NEWR', 'BUS', 'BACT', 'US', 'DPR', 'TEL', 'L...",930.0,1.000000,1.0,0.146444,0.228824,0.624732,0.624732,647.0
1,2007-02-01,2007-02-01 22:00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,-0.001860,...,2007-02-01 21:18:40,"{'NEWR', 'WWW', 'DRU', 'TEL', 'CHE', 'LEN'}",856.0,0.066667,0.0,0.014391,0.907705,0.077904,0.000000,190.0
2,2007-02-01,2007-02-01 22:00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,-0.078708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-02-01,2007-02-01 22:00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,0.014332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-02-01,2007-02-01 22:00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,-0.048613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007-02-01,2007-02-01 22:00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,0.012929,...,2007-02-01 06:07:27,"{'EUROPE', 'DK', 'NL', 'FR', 'DE', 'FI', 'WEU'...",112.0,0.447214,0.0,0.016462,0.861472,0.122067,0.000000,60.0
6,2007-02-01,2007-02-01 22:00:00,ABC.N,AmerisourceBergen Corp,1657300.0,52.37,52.40,-0.000191,0.008468,0.089000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007-02-01,2007-02-01 22:00:00,ABD.N,ACCO Brands Corp,1186200.0,23.63,24.13,-0.020721,-0.007404,0.005104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2007-02-01,2007-02-01 22:00:00,ABM.N,ABM Industries Inc,301200.0,26.19,25.90,0.013545,0.014890,0.068980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007-02-01,2007-02-01 22:00:00,ABT.N,Abbott Laboratories,5692300.0,52.87,52.50,-0.002453,-0.004739,0.001515,...,2007-02-01 22:15:04,"{'INS', 'WASH', 'DRU', 'HEA', 'US', 'RTRS', 'R...",99.0,0.353553,1.0,0.246808,0.341916,0.411276,0.411276,99.0


In [177]:
# Saving to csv

combined_df.to_csv('data/combined_data_v2.csv')

In [178]:
# Saving 2016 only to csv

combined2016_df = combined_df[(combined_df['date'] >= '2016-01-01') & (combined_df['date'] < '2017-01-01')]
combined2016_df.to_csv('data/combined2016_data_v2.csv')

## Further Data Aggregation

Upon using the data, we further realised that we needed to aggregate the news data by date for each day.  Since there are instances of multiple news surrounding a single company in a day, this made complicated calculations in Tableau.  Hence we aggregated the data and added a total number of news articles column, and averaged the values if there are more than one news entry per company per day.

In [5]:
combined_df = pd.read_csv('data/combined_data_v2.csv', index_col=0)

C:\Users\kw.UNLOVEDPC\Anaconda3\envs\py3-Env\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
combined2016_df = pd.read_csv('data/combined2016_data_v2.csv', index_col=0)

In [50]:
combined_df.head(30)

,date,timeMkt,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,...,timeNews,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
0,2007-02-01,2007-02-01 22:00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,-0.001860,...,2007-02-01 16:00:04,"{'NEWR', 'BUS', 'BACT', 'US', 'DPR', 'TEL', 'L...",930.0,1.000000,1.0,0.146444,0.228824,0.624732,0.624732,647.0
1,2007-02-01,2007-02-01 22:00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,-0.001860,...,2007-02-01 21:18:40,"{'NEWR', 'WWW', 'DRU', 'TEL', 'CHE', 'LEN'}",856.0,0.066667,0.0,0.014391,0.907705,0.077904,0.000000,190.0
2,2007-02-01,2007-02-01 22:00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,-0.078708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-02-01,2007-02-01 22:00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,0.014332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-02-01,2007-02-01 22:00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,-0.048613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007-02-01,2007-02-01 22:00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,0.012929,...,2007-02-01 06:07:27,"{'EUROPE', 'DK', 'NL', 'FR', 'DE', 'FI', 'WEU'...",112.0,0.447214,0.0,0.016462,0.861472,0.122067,0.000000,60.0
6,2007-02-01,2007-02-01 22:00:00,ABC.N,AmerisourceBergen Corp,1657300.0,52.37,52.40,-0.000191,0.008468,0.089000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007-02-01,2007-02-01 22:00:00,ABD.N,ACCO Brands Corp,1186200.0,23.63,24.13,-0.020721,-0.007404,0.005104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2007-02-01,2007-02-01 22:00:00,ABM.N,ABM Industries Inc,301200.0,26.19,25.90,0.013545,0.014890,0.068980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007-02-01,2007-02-01 22:00:00,ABT.N,Abbott Laboratories,5692300.0,52.87,52.50,-0.002453,-0.004739,0.001515,...,2007-02-01 22:15:04,"{'INS', 'WASH', 'DRU', 'HEA', 'US', 'RTRS', 'R...",99.0,0.353553,1.0,0.246808,0.341916,0.411276,0.411276,99.0


In [42]:
dfc2016mean = combined2016_df.groupby(['date','assetCode']).mean().reset_index()
dfc2016mean.shape

(458863, 18)

In [43]:
dfc2016sum = combined2016_df.groupby(['date','assetCode']).sum().reset_index()
dfc2016sum.shape

(458863, 18)

In [44]:
dfc2016count = combined2016_df.groupby(['date','assetCode']).timeNews.count().reset_index()
dfc2016count.shape

(458863, 3)

In [46]:
a0 = combined2016_df[combined2016_df.date == '2016-10-17']
a0[a0.assetCode == 'NFLX.O']

,date,timeMkt,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,...,timeNews,subjects,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
7896082,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 13:00:44,"{'BLR', 'REAL', 'SWIT', 'ITSE', 'REIT', 'CYCS'...",22.0,1.0,0.0,0.164653,0.613988,0.221360,0.000000,22.0
7896083,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 14:49:35,"{'BLR', 'SWIT', 'STX', 'ITSE', 'CYCS', 'HOT', ...",17.0,1.0,-1.0,0.590178,0.268349,0.141473,-0.590178,17.0
7896084,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 17:30:10,"{'SWIT', 'STX', 'ITSE', 'INDU', 'OPT', 'CYCS',...",212.0,1.0,-1.0,0.779074,0.148168,0.072758,-0.779074,135.0
7896085,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 17:55:26,"{'BLR', 'COEN', 'MRG', 'RGEN', 'FOBE', 'HECA',...",1593.0,1.0,-1.0,0.420607,0.305193,0.274200,-0.420607,139.0
7896086,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:01,"{'SWIT', 'ENTS', 'ITSE', 'CYCS', 'NEWR', 'BACT...",319.0,1.0,1.0,0.137028,0.390973,0.471998,0.471998,315.0
7896087,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:01,"{'BLR', 'SWIT', 'ITSE', 'CYCS', 'BACT', 'TMT',...",9.0,1.0,0.0,0.135848,0.604001,0.260151,0.000000,9.0
7896088,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:02,"{'SWIT', 'STX', 'ITSE', 'INDU', 'CYCS', 'BACT'...",9.0,1.0,-1.0,0.819144,0.125228,0.055628,-0.819144,9.0
7896089,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:29,"{'BLR', 'SWIT', 'ITSE', 'CYCS', 'ASIA', 'RESF'...",16.0,1.0,-1.0,0.696831,0.111872,0.191297,-0.696831,16.0
7896090,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:30,"{'BLR', 'SWIT', 'ITSE', 'CYCS', 'ASIA', 'RESF'...",17.0,1.0,1.0,0.031098,0.118643,0.850259,0.850259,17.0
7896091,2016-10-17,2016-10-17 22:00:00,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,...,2016-10-17 20:05:38,"{'BLR', 'SWIT', 'ITSE', 'CYCS', 'ASIA', 'RESF'...",14.0,1.0,0.0,0.157039,0.614943,0.228018,0.000000,14.0


In [45]:
a1 = dfc2016mean[dfc2016mean.date == '2016-10-17']
a1[a1.assetCode == 'NFLX.O']

,date,assetCode,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,returnsOpenPrevRaw10,universe,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
362736,2016-10-17,NFLX.O,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,0.02551,1.0,101.25,1.0,0.210526,0.212532,0.408857,0.378611,0.13159,67.592105


In [33]:
a2 = dfc2016sum[dfc2016sum.date == '2016-10-17']
a2[a2.assetCode == 'NFLX.O']

,date,assetCode,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,returnsOpenPrevRaw10,universe,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
362736,2016-10-17,NFLX.O,2.020801e+09,7584.8,7638.0,-1.250813,-0.815435,-2.095684,1.938776,76.0,7695.0,76.0,16.0,16.152462,31.073142,28.774399,10.000865,5137.0


In [40]:
a3 = dfc2016count[dfc2016count.date == '2016-10-17']
a3[a3.assetCode == 'NFLX.O']

,date,assetCode,timeNews
362736,2016-10-17,NFLX.O,76


In [35]:
b = combined2016_df[combined2016_df.date == '2016-10-17']
b[b.assetCode == 'NFLX.O'].wordCount.sum()

7695.0

In [63]:
# Getting groupby objects and getting the 
dfc2016mean = combined2016_df.groupby(['date','assetCode', 'assetName']).mean().reset_index()
dfc2016sum = combined2016_df.groupby(['date','assetCode', 'assetName']).sum().reset_index()
dfc2016count = combined2016_df.groupby(['date','assetCode', 'assetName']).timeNews.count().reset_index()

# Using mean as the basis to start since most fields mean is the most relevant
dfc2016 = dfc2016mean.copy(deep=True)

# inserting new field number of news articles
dfc2016.insert(loc=dfc2016.columns.get_loc('universe')+1, 
                   column='newsArticles', 
                   value=dfc2016count['timeNews'])

# Changing the following mean values to sum
dfc2016['wordCount'] = dfc2016sum['wordCount']
dfc2016['sentimentWordCount'] = dfc2016sum['sentimentWordCount']

# Re-evaluting the sentimentRating
dfc2016.loc[(dfc2016.sentimentClass < 0), 'sentimentRating'] = -1* dfc2016[dfc2016.sentimentClass < 0].sentimentNegative
dfc2016.loc[(dfc2016.sentimentClass == 0), 'sentimentRating'] = 0
dfc2016.loc[(dfc2016.sentimentClass > 0), 'sentimentRating'] = dfc2016[dfc2016.sentimentClass > 0].sentimentPositive

In [64]:
# Checking...
aX = dfc2016[dfc2016.date == '2016-10-17']
aX[aX.assetCode == 'NFLX.O']

,date,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,returnsOpenPrevRaw10,universe,newsArticles,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
362736,2016-10-17,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,0.02551,1.0,76,7695.0,1.0,0.210526,0.212532,0.408857,0.378611,0.378611,5137.0


In [65]:
# Getting groupby objects and getting the 
dfcmean = combined_df.groupby(['date','assetCode', 'assetName']).mean().reset_index()
dfcsum = combined_df.groupby(['date','assetCode', 'assetName']).sum().reset_index()
dfccount = combined_df.groupby(['date','assetCode', 'assetName']).timeNews.count().reset_index()

# Using mean as the basis to start since most fields mean is the most relevant
dfc = dfcmean.copy(deep=True)

# inserting new field number of news articles
dfc.insert(loc=dfc.columns.get_loc('universe')+1, 
                   column='newsArticles', 
                   value=dfccount['timeNews'])

# Changing the following mean values to sum
dfc['wordCount'] = dfcsum['wordCount']
dfc['sentimentWordCount'] = dfcsum['sentimentWordCount']

# Re-evaluting the sentimentRating
dfc.loc[(dfc.sentimentClass < 0), 'sentimentRating'] = -1* dfc[dfc.sentimentClass < 0].sentimentNegative
dfc.loc[(dfc.sentimentClass == 0), 'sentimentRating'] = 0
dfc.loc[(dfc.sentimentClass > 0), 'sentimentRating'] = dfc[dfc.sentimentClass > 0].sentimentPositive

In [68]:
# Checking...
bX = dfc[dfc.date == '2016-10-17']
bX[bX.assetCode == 'NFLX.O']

,date,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevRaw10,returnsOpenPrevRaw10,universe,newsArticles,wordCount,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRating,sentimentWordCount
3976829,2016-10-17,NFLX.O,Netflix Inc,26589483.0,99.8,100.5,-0.016458,-0.010729,-0.027575,0.02551,1.0,76,7695.0,1.0,0.210526,0.212532,0.408857,0.378611,0.378611,5137.0


In [69]:
# Saving to csv

dfc.to_csv('data/combined_data_v3.csv')
dfc2016.to_csv('data/combined2016_data_v3.csv')